# Agent Supervisor

The [previous example](multi-agent-collaboration.ipynb) routed messages automatically based on the output of the initial researcher agent.

We can also choose to use an LLM to orchestrate the different agents.

Below, we will create an agent group, with an agent supervisor to help delegate tasks.

![diagram](./img/supervisor-diagram.png)

To simplify the code in each agent node, we will use the AgentExecutor class from LangChain. This and other "advanced agent" notebooks are designed to show how you can implement certain design patterns in LangGraph. If the pattern suits your needs, we recommend combining it with some of the other fundamental patterns described elsewhere in the docs for best performance.

Before we build, let's configure our environment:

In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain langchain_openai langchain_experimental langsmith pandas

In [2]:
!pip install langchain-google-genai
!pip install google-generativeai
!pip install langchain-groq
from langchain.agents import create_openai_functions_agent
from langchain_google_genai import ChatGoogleGenerativeAI


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.5 MB/s eta 0:00:00


In [3]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("GOOGLE_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")
_set_if_undefined("GROQ_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"


Please provide your GOOGLE_API_KEY··········
Please provide your LANGCHAIN_API_KEY··········
Please provide your TAVILY_API_KEY··········
Please provide your GROQ_API_KEY··········


## Create tools

For this example, you will make an agent to do web research with a search engine, and one agent to create plots. Define the tools they'll use below:

In [4]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()

## Helper Utilities

Define a helper function that we will use to create the nodes in the graph - it takes care of converting the agent response to a human message. This is important because that is how we will add it the global state of the graph

In [42]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [43]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

### Create Agent Supervisor

It will use function calling to choose the next worker node OR finish processing.

In [44]:
from langchain.tools.render import format_tool_to_openai_function
from langchain_groq import ChatGroq

In [45]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_openai import ChatOpenAI
# from pydantic import BaseModel
# from typing import Literal

# members = ["Researcher", "Coder"]
# system_prompt = (
#     "You are a supervisor tasked with managing a conversation between the"
#     " following workers:  {members}. Given the following user request,"
#     " respond with the worker to act next. Each worker will perform a"
#     " task and respond with their results and status. When finished,"
#     " respond with FINISH."
# )
# options = ["FINISH"] + members

# class routeResponse(BaseModel):
#     next: Literal["FINISH", "Researcher", "Coder"]

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         MessagesPlaceholder(variable_name="messages"),
#         (
#             "system",
#             "Given the conversation above, who should act next?"
#             " Or should we FINISH? Select one of: {options}",
#         ),
#     ]
# ).partial(options=", ".join(options), members=", ".join(members))

# llm = ChatGroq(
#     model="llama-3.1-70b-versatile",  # Alternative model
#     temperature=0,
#     max_tokens=None
# )

# def supervisor_agent(state):
#     supervisor_chain = (
#         prompt
#         | llm.with_structured_output(routeResponse)
#     )
#     return supervisor_chain.invoke(state)


# Create Agent Supervisor
from langchain.tools.render import format_tool_to_openai_function
from langchain_groq import ChatGroq
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

options = ["FINISH"] + members

function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatGroq(
    model="llama-3.1-70b-versatile",  # Alternative model
    temperature=0,
    max_tokens=None
)

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [46]:
# Helper function for JSON serialization
def serialize_messages(messages):
    # Convert non-serializable objects to strings
    return [
        msg.dict() if isinstance(msg, HumanMessage) else str(msg)
        for msg in messages
    ]

## Construct Graph

We're ready to start building the graph. Below, define the state and worker nodes using the function we just defined.

In [52]:
# Construct Graph
import functools
import operator
from typing import Sequence, TypedDict

from langgraph.graph import END, StateGraph, START

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

# Create research and code agents
research_agent = create_agent(llm, [tavily_tool], "You are a web researcher.")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

code_agent = create_agent(
    llm,
    [python_repl_tool],
    """You are a helpful AI assistant that writes safe, executable Python code.
You may generate Python code to analyze data and generate charts using matplotlib.
Always make sure the code is correctly formatted.
Ensure all code is wrapped within a single code block, delimited by triple backticks.
For example:
python print('Hello world!')
"""
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

# Build the workflow graph
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)




Now connect all the edges in the graph.

In [53]:
# Connect edges in the graph
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

## Invoke the team

With the graph created, we can now invoke it and see how it performs!

In [54]:
# Invoke the team - First case
serialized_messages = serialize_messages([
    HumanMessage(content="Print hello world to the terminal")
])

for s in graph.stream({"messages": serialized_messages}):
    if "__end__" not in s:
        print(s)
        print("----")


{'supervisor': {'next': 'Coder'}}
----


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=Python_REPL>{"query": "print("}hello world!{"query": ")", "name": "Python_REPL"} </function>'}}

In [55]:
# Invoke the team - Second case
serialized_messages = serialize_messages([
    HumanMessage(content="Write a brief research report on pikas.")
])

for s in graph.stream({"messages": serialized_messages}, {"recursion_limit": 100}):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='Pikas are small mammals that live in mountainous regions and are well adapted to cold temperatures. They have a unique way of surviving the winter by collecting and storing food in piles, called "haypiles," which they use as a food source during the winter months. Pikas are also known for their ability to live in high-altitude areas with limited vegetation, and they play an important role in their ecosystems as "ecosystem engineers." However, pikas are facing threats due to climate change, which is causing their populations to decline. They are sensitive to high temperatures and are moving to higher elevations in search of cooler temperatures, but this is not a sustainable solution as there is no higher territory for them to escape to. As a result, pikas are considered to be one of the best early warning systems for detecting global warming in the western United States.', name='Researcher')]}